In [121]:
import sys, os, time
import redis
import pickle
import numpy as np
import random
import foundation.util as util

In [2]:
root = '/home/fleeb/workspace/ml_datasets/wiki/'

In [87]:
def random_str(N):
    return np.random.randint(256, size=N, dtype=np.uint8).tobytes()

In [155]:
db = redis.Redis(db=0)

In [158]:
db.keys()

[]

In [147]:
data = db.hgetall('test')

In [157]:
len(data)

100

In [152]:
IDs = set()

In [154]:
db.memory_usage('test')

110080

In [10]:
db.get('q')

b'4,5,6'

In [5]:
host, port = b'localhost', 6379

In [162]:
def init_worker(host, port, db_num=0, **other_args):
    out = {
        'db': redis.Redis(host=host, port=port, db=db_num),
        'IDdb': redis.Redis(host=host, port=port, db=0),
    }
    return out

def work(pid, key, value, db, **other_args):
    
    return db.sadd(key, value)

def work2(pid, key, value, db, IDdb, **other_args):
    
    pi = IDdb
    db.sadd(ID, value)
    
    
    return 


In [163]:
N = 100
num_workers = 4

In [198]:
keys = [
    random_str(50) for _ in range(100)
]
values = [
    random_str(1000) for _ in range(1000)
]

def get_next():
    while True:
        yield {'key':random_str(1000), 'value':random_str(1000)}

In [211]:
{k:v for k,v in db.info('memory').items() if 'human' in k}

{'maxmemory_human': '0B',
 'total_system_memory_human': '11.68G',
 'used_memory_human': '217.05M',
 'used_memory_lua_human': '37.00K',
 'used_memory_peak_human': '217.11M',
 'used_memory_rss_human': '227.36M'}

In [200]:
private_args = {
    'host':host,
    'port':port,
    'db_num':1,
}

shared_args = {
    
}

unique_args = [{'pid':i} for i in range(num_workers)]

volatile_args = iter(get_next())

In [207]:
start = time.time()
creator = util.Farmer(fn=work, init_fn=init_worker, auto_dispatch=False,
                      unique_worker_args=unique_args, volatile_gen=volatile_args,
                      private_args=private_args, shared_args=shared_args,
                      num_workers=num_workers,
        )
creator._dispatch(100000)

for _ in creator:
    pass
total = time.time() - start

In [209]:
print(total)
print(db.info('memory')['used_memory_human'], db.info('keyspace')['db1']['keys'])

6.430676221847534
217.03M 100000


In [210]:
print(total)
print(db.info('memory')['used_memory_human'], db.info('keyspace')['db1']['keys'])

6.430676221847534
217.03M 100000


In [17]:

_inc_temp = '''UPDATE articles SET num = num + 1 WHERE name IN ({})'''
_insert_temp = '''INSERT INTO articles VALUES (?,1);'''
_sel_id_temp = '''SELECT rowid FROM articles WHERE name IN ({})'''
_sel_name_temp = '''SELECT name FROM articles WHERE rowid IN ({})'''
_sel_full_id_temp = '''SELECT * FROM articles WHERE rowid IN ({})'''
_sel_full_name_temp = '''SELECT * FROM articles WHERE name IN ({})'''

class Articles(object):
    def __init__(self, db_path, lock=None):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.lock = None
        
    def names(self):
        self.cursor.execute('''SELECT name FROM articles''')
        return iter(self.cursor)
    
    def nums(self):
        self.cursor.execute('''SELECT name FROM articles''')
        return iter(self.cursor)
    
    def items(self):
        self.cursor.execute('''SELECT * FROM articles''')
        return iter(self.cursor)
        
    def add(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        for name in names:
            try:
                #print(_insert_temp, name)
                self.cursor.execute(_insert_temp, (name,))
            except sqlite3.IntegrityError:
                pass
        #self.cursor.executemany(_insert_temp, [(n,) for n in names])
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def increment(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        self.cursor.execute(_inc_temp.format(', '.join('?'*len(names))), names)
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def __len__(self):
        self.cursor.execute('''SELECT COUNT(*) FROM articles''')
        return self.cursor.fetchall()[0][0]
        
    def get_id(self, *names):
        self.cursor.execute(_sel_id_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
        
    def get_name(self, *IDs):
        self.cursor.execute(_sel_name_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def get_full_from_names(self, *names):
        self.cursor.execute(_sel_full_name_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
    
    def get_full_from_ids(self, *IDs):
        self.cursor.execute(_sel_full_id_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def __del__(self):
        self.conn.close()

In [18]:
db = Articles('example.db')

In [19]:
list(db.items())

[('t1', 6), ('t2', 4), ('t3', 1)]

In [20]:
len(db)

3

In [21]:
db.add('t1', 't3')

In [22]:
len(db)

3

In [23]:
db.increment('t2')

In [24]:
list(db.items())

[('t1', 6), ('t2', 5), ('t3', 1)]